# Exploring Hacker News Posts

In this project, we'll work with a dataset of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).


Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") receive votes and comments, similar to reddit.

Below are descriptions of the columns:

- `id`: the unique identifier from Hacker News for the post
- `title`: the title of the post
- `url`: the URL that the post links to, if the post has a URL
- `num_points`: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: the number of comments on the post
- `author`: the username of the person who submitted the post
- `created_at`: the date and time of the post's submission

## Opening and Exploring the Data

We're specifically interested in posts with titles that begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just something interesting.

We'll compare these two types of posts to determine the following:
* Do `Ask HN` or `Show HN` receive more comments on average?
* Do posts created at a certain time receive more comments on average?

Let's start by importing the libraries we need and reading the dataset into a list of lists.

In [3]:
from csv import reader

opened_file = open(r"C:\Users\youss\Desktop\HN_posts_year_to_Sep_26_2016.csv", encoding='unicode_escape')
read_file = reader(opened_file)

hn = list(read_file)
headers = hn[0]
hn = hn[1:]
print(headers)
print()

for row in hn[:3]:
    print(row, end='\n\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26']

['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']

['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']



## Extracting Ask HN and Show HN Posts

Now that we've removed the headers from `hn`, we're ready to filter our data. Since we're only concerned with post titles beginning with `Ask HN` or `Show HN`, we'll create new lists of lists containing just the data for those titles.

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn') == True:
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))
for row in ask_posts[:3]:
    print(row, end='\n\n')

9139
10158
273824
['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53']

['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17']

['12577908', 'Ask HN: How a DNS problem can be limited to a geographic region?', '', '1', '0', 'kuon', '9/25/2016 22:57']



## Calculating the Average Number of Comments for Ask HN and Show HN Posts


Next, let's determine if ask posts or show posts receive more comments on average.

In [5]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = row[4]
    total_ask_comments += int(num_comments)
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

10.393478498741656


In [6]:
total_show_comments = 0

for row in show_posts:
    total_show_comments += int(row[4])
    
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

4.886099625910612


On average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Number of Ask Posts and Comments by Hour Created

We'll determine if ask posts created at a certain time are more likely to attract comments. First, let's calculate the number of ask posts created per hour, along with the total number of comments.

In [7]:
import datetime as dt

counts_by_hour = {}
comments_by_hour = {}

for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    date = dt.datetime.strptime(created_at, '%m/%d/%Y %H:%M')
    hour = dt.datetime.strftime(date, "%H")
    
    if hour not in counts_by_hour:  # or if hour not in comments_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments

## Calculating the Average Number of Comments for Ask HN Posts by Hour

Now, we'll use the dictionaries `counts_by_hour` and `comments_by_hour` to calculate the average number of comments for posts created during each hour of the day.

In [8]:
avg_by_hour = []

for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])

print(avg_by_hour)

[['02', 11.137546468401487], ['01', 7.407801418439717], ['22', 8.804177545691905], ['21', 8.687258687258687], ['19', 7.163043478260869], ['17', 9.449744463373083], ['15', 28.676470588235293], ['14', 9.692007797270955], ['13', 16.31756756756757], ['11', 8.96474358974359], ['10', 10.684397163120567], ['09', 6.653153153153153], ['07', 7.013274336283186], ['03', 7.948339483394834], ['23', 6.696793002915452], ['20', 8.749019607843136], ['16', 7.713298791018998], ['08', 9.190661478599221], ['00', 7.5647840531561465], ['18', 7.94299674267101], ['12', 12.380116959064328], ['04', 9.7119341563786], ['06', 6.782051282051282], ['05', 8.794258373205741]]


Although we now have the results we need, this format makes it difficult to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

## Sorting and Printing Values from a List of Lists


In [9]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments", end='\n\n')
for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1], '%H').strftime('%H:%M')
    average = row[0]
    print("{}: {:.2f} average comments per post".format(hour, average))

Top 5 Hours for Ask Posts Comments

15:00: 28.68 average comments per post
13:00: 16.32 average comments per post
12:00: 12.38 average comments per post
02:00: 11.14 average comments per post
10:00: 10.68 average comments per post


## Conclusion

That's it for our project! Here's a quick summary of what we accomplished so far:

* We set a goal for the project.
* We collected and sorted the data.
* We reformatted and cleaned the data to prepare it for analysis.
* We analyzed the data.